# Charging Model

## initialize set
- import modules
- set problems
- set idnex
- define variables
- read data from files
- define constants
- handle the relationship between variables 

In [1]:
# 导入模块
import xpress as xp
import numpy as np
import pandas as pd

In [2]:
# Create a problem called myproblem

prob = xp.problem(name='charging')

In [3]:
# Set index
number_of_types = 3 # i: 1..3
types = range(number_of_types)

number_of_years = 4 # Time t: 1..4 0..4
years = range(number_of_years)
number_of_years2 = number_of_years + 1
years2 = range(number_of_years2)

number_of_grids = 434 # grid g: 1..434
grids = range(number_of_grids)


In [4]:
# Define our decision variable as a numpy array
# 设置变量
x1 = np.array([xp.var( name='x1_{0}_{1}_{2}'.format(i+1,g+1,t+1),vartype = xp.integer)
                    for i in types for g in grids for t in years], dtype=xp.npvar).reshape(number_of_types,number_of_grids,number_of_years)
x2 = np.array([xp.var( name='x2_{0}_{1}_{2}'.format(i+1,g+1,t+1),vartype = xp.integer)
                    for i in types for g in grids for t in years], dtype=xp.npvar).reshape(number_of_types,number_of_grids,number_of_years)
x = np.array([xp.var( name='x_{0}_{1}_{2}'.format(i+1,g+1,t),vartype = xp.integer)
                    for i in types for g in grids for t in range(number_of_years2)], dtype=xp.npvar).reshape(number_of_types,number_of_grids,number_of_years2)
# x = x1 + x2
z = np.array([xp.var( name='z_{0}_{1}'.format(g+1,t+1),vartype = xp.binary)
                    for g in grids for t in years], dtype=xp.npvar).reshape(number_of_grids,number_of_years)

y = np.array([xp.var( name='y_{0}_{1}'.format(g+1,t+1),vartype = xp.integer)
                    for g in grids for t in years], dtype=xp.npvar).reshape(number_of_grids,number_of_years)
# y_{g,t} = \sum_{i = 1..3} (x_{i,g,0} + \sum_{t<t}x_{i,g,t})
# y: report3公式写的有点混乱，应该是在t年g方块所有的chrger
prob.addVariable(x1,x2,x,y,z)

In [5]:
# read data from files
pot_char_poi = pd.read_excel('Project_data/Potential_charging_points.xlsx')
intere_poi = pd.read_excel('Project_data/Interest _points.xlsx')
demand = pd.read_excel('Project_data/Demand_data.xlsx')
char_poi =  pd.read_excel('Project_data/Charging_points.xlsx')


In [6]:
#  Constant
# 未打折的成本价格
c = np.array(types) # building cost for charging point of type i
c = [1,1,1] # 可以修改

# 打折的成本价格
r = np.array(types) # the reduced building cost for charging point of type i
r = [0.8,0.8,0.8] # 可以修改

# 每个grid的location数量限制
B = np.array(grids) # the maximal number of location in the grid square g
for i in B:
    i = 3 # 可以修改
    
# 每location的charger上线
A = 5 # 可以修改

# A*B_{g}:the maximal number of charging points in the grid square g, where 

# 每种charger的最低比例要求
L = np.array(types) # the lower bound of proportion for type i charging point
L = [0.3,0.3,0.3] # 可以修改

# 每种charger的最高限制
U = np.array(types) # the Upper bound of proportion for type i charging point ????
U = [0.7,0.7,0.7] # 可以修改

# Mit = np.array(types) # the higher bound of proportion for type i charging point
# Mit = [0.9,0.9,0.9]

# 市中心的charger上限
SP = 100 # maximal number of station points in the centre of city.

# 总需求最低满足的限度
TS = 0.9 # the satisfaction proportion of for the total demand

# 被邻居和自己满足的需求下限
MS = 0.8 # the proportion of for the demand provided by neighbor.

# t年g方块的需求
d = np.array((demand['Demand_0'],demand['Demand_1'],demand['Demand_2'],demand['Demand_3']))
# demand for the grid g in year t

# 每种charger提供的最低电量
Pi1 = np.array(years) # the minimum demand type i can provide per year
Pi1 = [2000,4000,30000]

# 每种charger提供的最高电量
Pi = np.array(years) # the maximum demand type i can provide per year 
Pi = [3500,5200,55000]

# the number of charger point of type i which has been already established in the grid g
char_poi['Type'].replace('Slow',0,inplace = True)
char_poi['Type'].replace('Fast',1,inplace = True)
char_poi['Type'].replace('Rapid',2,inplace = True)
exist_charger = pd.DataFrame({'Type':char_poi['Type'],'Grid':char_poi['grid number']})
w = np.zeros([number_of_types,number_of_grids])
for g in grids:
    if (g+1) in exist_charger['Grid']:
        for i in types:
            w[i][g] = exist_charger[(exist_charger.Type == i)&(exist_charger.Grid == (g+1))].count()[0]

# 每个grid的potential location数量
# Pg: the number of potential locations in the grid square
pg = (pot_char_poi['grid number'].value_counts())
Pg = np.zeros(number_of_grids) 
for i in pg.index:
    Pg[i-1] = pg[i]

In [7]:
# relastion between variables and exist constants
# 处理变量基本关系
# x_{i,g,0}
for i in types:
    for g in grids:
        if g+1 in list(char_poi['grid number']):
            prob.addConstraint( x[i,g,0] == w[i,g])
        else:
            prob.addConstraint(x[i,g,0] == 0)

## Add Constraints

In [8]:
# constrain 1 
# 每年每个grid的charger不能超过限制

prob.addConstraint(y[g,t] <= A*B[g] for g in grids for t in years)

In [9]:
# constrian 2 限定市中心的charger数量
# 市中心的定义方法：需求大于1000
DCenter = 1000
grid_center = np.zeros((number_of_years,number_of_grids))
for t in years:
    grid_center_y = []
    path = str('Demand_' + str(t))
    grid_center_y = demand[demand[path]>=1000].index.tolist()
    for i in range(len(grid_center_y)):
        grid_center[t][i] = grid_center_y[i]
    print('the ',t+1,' year, city center\'s grid numbers',len(grid_center_y))

# 这里注意，有个问题，不能让SP等于一个恒定的值，因为市中心范围有变化
prob.addConstraint((xp.Sum(y[int(g),t] for g in grid_center[t]) <= SP) for t in years)  


the  1  year, city center's grid numbers 21
the  2  year, city center's grid numbers 42
the  3  year, city center's grid numbers 49
the  4  year, city center's grid numbers 65


In [10]:
# constrain 3
# 每种类型的charger满足最高和最低的限制（charger多样化）
for i in types:
    for t in years2:
        if t>0:
            prob.addConstraint(L[i]*xp.Sum(x[i1,g,t1] for i1 in types for g in grids for t1 in years2 if t1<=t) <= 
                          xp.Sum(x[i,g,t1] for g in grids for t1 in years2 if t1 <=t))
            prob.addConstraint(U[i]*xp.Sum(x[i1,g,t1] for i1 in types for g in grids for t1 in years2 if t1<=t) >= 
                          xp.Sum(x[i,g,t1] for g in grids for t1 in years2 if t1<=t))


In [11]:
# constrain 4
# 总需求至少满足TS的范围

# for t in years2:
#     if t>0:
#         prob.addConstraint(TS*np.sum(d[:][t-1]) 
# #                            <= ((2*Pi1[i]*xp.Sum(x[i,g,t1] for g in grids for t1 in years2 if t1<=t))for i in types))
TotalDemand = np.array([np.sum(d[:][0]),np.sum(d[:][1]),np.sum(d[:][2]),np.sum(d[:][3])])
prob.addConstraint(2*xp.Sum(Pi1[i]*xp.Sum(x[i,g,t1] for g in grids) 
                          for i in types for t1 in years2 if t1<=t)>=TS*TotalDemand[t-1] for t in years2 if t>=1)


In [12]:
# constraint 5
# 被邻居满足的
neighbors = demand['NEIGHBORS']
for g in grids:
    neighbor = np.array(neighbors[g].strip('[]').split(','),dtype = 'int')
    neighbor = np.insert(neighbor,0,[int(demand['Ref'][g])])
#     prob.addConstraint(2*xp.Sum(Pi1[i]*xp.Sum(x[i,g1,t1] for g1 in neighbor) 
#                           for i in types for t1 in years2 if t1<=t)>=MS*d[0][g] for t in years2 if t>=1)
    prob.addConstraint(xp.Sum(Pi1[i]*xp.Sum(x[i,n-1,t1] for n in neighbor) for i in types for t1 in years2 if t1<=t)
                      >= 0 for t in years2 if t >=1)



In [13]:
# constraint 6
# x = x1 + x2
prob.addConstraint(x[i,g,t+1] == x1[i,g,t] + x2[i,g,t] for i in types for g in grids for t in years)

In [14]:
# constraint 7
# x and y
# for g in grids:
#     prob.addConstraint(y[g,0] == xp.Sum((x[i,g,0]+x[i,g,1]) for i in types))
#     prob.addConstraint(y[g,1] == y[g,0] + xp.Sum((x[i,g,2]) for i in types))
#     prob.addConstraint(y[g,2] == y[g,1] + xp.Sum((x[i,g,3]) for i in types))
#     prob.addConstraint(y[g,3] == y[g,2] + xp.Sum((x[i,g,4]) for i in types))

for t in years2:
    if t >0:
        for g in grids:
            prob.addConstraint(y[g,t-1] == xp.Sum(x[i,g,t1] for i in types for t1 in years if t1<=t ))
#     else:
#         for g in grids:
#             prob.addConstraint(y[g,t] == 0 )

In [15]:
# constraint 8
M = 1000
        
prob.addConstraint((1-z[g,t]) <= y[g,t] for g in grids for t in years)
prob.addConstraint(y[g,t] <= (1-z[g,t])*M for g in grids for t in years)
prob.addConstraint(x1[i,g,t] <= M*z[g,t] for i in types for g in grids for t in years)
prob.addConstraint(x2[i,g,t] <= M*(1-z[g,t]) for i in types for g in grids for t in years)


In [16]:
# constraint 9
# 定义变量的时候已经写了

In [17]:
# constraint 10
# >= 0
prob.addConstraint(x1[i,g,t] >= 0 for i in types for g in grids for t in years)
prob.addConstraint(x2[i,g,t] >= 0 for i in types for g in grids for t in years)
prob.addConstraint(y[g,t] >= 0 for g in grids for t in years)

## 问题解决

In [18]:
# Objective function
prob.setObjective(xp.Sum(c[i]*x1[i,g,t] for i in types for g in grids for t in years) 
                  + xp.Sum(r[i] * x2[i,g,t] for i in types for g in grids for t in years),
                 sense = xp.minimize)


In [19]:
prob.write("problem","lp")

In [20]:
prob.solve()

FICO Xpress v8.13.7, Hyper, solve started 11:43:01, Nov 28, 2022
Heap usage: 22MB (peak 22MB, 5316KB system)
Minimizing MILP charging using up to 16 threads, with these control settings:
OUTPUTLOG = 1
Original problem has:
     37790 rows        20398 cols       357909 elements     20398 globals
Presolved problem has:
      9932 rows         8629 cols        88114 elements      8629 globals
LP relaxation tightened
Presolve finished in 0 seconds
Heap usage: 25MB (peak 51MB, 5316KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 3.00e-01,  6.00e+04] / [ 1.95e-03,  1.99e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  9.40e+05] / [ 1.00e-01,  2.00e+03]
  Objective      [min,max] : [ 8.00e-01,  1.00e+00] / [ 8.00e-01,  1.00e+00]
Autoscaling applied standard scaling

Symmetric problem: generators: 263, support set: 5405
 Number of orbits: 158, largest orbit: 202
 Row orbits: 182, row support: 6218
Will try to keep branch and

In [ ]:
print(prob.getObjVal())

26.40000000000001


## 过往草稿

In [ ]:
#中心区域数量限制
distance = demand['Distance from Centre'].copy()
distance.sort_values(inplace = True)
center = distance[distance <= 1000]
center_grid = np.array(center.index)
print(center_grid)

In [ ]:
M = 10000

In [ ]:
for t in years:
    if t >0:
        number_center = 0
        for g in center_grid:
            number_center += xp.Sum(x[i,g-1,t1] for i in types for t1 in years if t1<= t )
        prob.addConstraint(number_center <= 100)


In [ ]:
B = Pg*5                          #需要求
print(B)


In [ ]:
# electricity demand 请问G‘怎么定义,我很迷茫，我暂时定义为他的邻居吧。。。。。。。。。。还有为什么突然有了个t‘，我很迷茫
# 这里的formulation文件有两个公式，我看着好像是一个意思？代码部分写的是第一个公式
neighbors = demand['NEIGHBORS']

for g in grids:
    neighbor = np.array(neighbors[g].strip('[]').split(','),dtype = 'int')
    for t in years:
        if t>0:
            prob.addConstraint(xp.Sum(Pi1[i]*x[i,n-1,t1] for i in types for n in neighbor for t1 in years if t1<=t) >= 0.8*Dgt[g,t-1])
for t in years:
    if t>0:
        prob.addConstraint(Pi1[i]*xp.Sum(x[i,g,t] for i in types for g in grids for t1 in years if t1<=t)>= 0.8*xp.Sum(Dgt[g,t-1] for g in grids))
        

In [ ]:
# interest point
# 得到每个点interest point数
M2 = 5
M3 = 1000
IG = intere_poi['grid number'].value_counts()
IG2 = np.zeros(number_of_grids)
for i in IG.index:
    IG2[i-1] = IG[i]
IG2
for t in years:
    if t>0:
        for g in grids:
         neighbor = np.array(neighbors[g].strip('[]').split(','),dtype = 'int')
         for i in neighbor:
            prob.addConstraint(xp.Sum(IG2[g1-1] for g1 in neighbor) >= z[g,t]*M2)


    

